In [1]:
import pandas as pd

providers     = pd.read_csv('providers_data.csv')
receivers     = pd.read_csv('receivers_data.csv')
food_listings = pd.read_csv('food_listings_data.csv')
claims        = pd.read_csv('claims_data.csv')

In [2]:
providers.head()

,Provider_ID,Name,Type,Address,City,Contact
0,1,Gonzales-Cochran,Supermarket,"74347 Christopher Extensions\nAndreamouth, OK ...",New Jessica,+1-600-220-0480
1,2,"Nielsen, Johnson and Fuller",Grocery Store,"91228 Hanson Stream\nWelchtown, OR 27136",East Sheena,+1-925-283-8901x6297
2,3,Miller-Black,Supermarket,"561 Martinez Point Suite 507\nGuzmanchester, W...",Lake Jesusview,001-517-295-2206
3,4,"Clark, Prince and Williams",Grocery Store,"467 Bell Trail Suite 409\nPort Jesus, IA 61188",Mendezmouth,556.944.8935x401
4,5,Coleman-Farley,Grocery Store,"078 Matthew Creek Apt. 319\nSaraborough, MA 53978",Valentineside,193.714.6577


In [3]:
receivers.head()

,Receiver_ID,Name,Type,City,Contact
0,1,Donald Gomez,Shelter,Port Carlburgh,(955)922-5295
1,2,Laurie Ramos,Individual,Lewisburgh,761.042.1570
2,3,Ashley Mckee,NGO,South Randalltown,691-023-0094x856
3,4,Erika Rose,NGO,South Shaneville,8296491111
4,5,John Romero,Individual,Bakerport,067.491.0154


In [4]:
food_listings.head()

,Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type
0,1,Bread,43,3/17/2025,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast
1,2,Soup,22,3/24/2025,791,Grocery Store,West James,Non-Vegetarian,Dinner
2,3,Fruits,46,3/28/2025,478,Catering Service,Lake Regina,Vegan,Breakfast
3,4,Fruits,15,3/16/2025,930,Restaurant,Kellytown,Vegan,Lunch
4,5,Soup,14,3/19/2025,279,Restaurant,Garciaport,Vegan,Dinner


In [5]:
claims.head()

,Claim_ID,Food_ID,Receiver_ID,Status,Timestamp
0,1,164,908,Pending,3/5/2025 5:26
1,2,353,391,Cancelled,3/11/2025 10:24
2,3,626,492,Completed,3/21/2025 0:59
3,4,61,933,Cancelled,3/4/2025 9:08
4,5,345,229,Pending,3/14/2025 15:17


In [6]:
import pandas as pd

food_listings['Expiry_Date'] = pd.to_datetime(food_listings['Expiry_Date'], format='%m/%d/%Y')
claims['Timestamp'] = pd.to_datetime(claims['Timestamp'], format='%m/%d/%Y %H:%M')

print(food_listings.dtypes)
print("="*50)
print(claims.dtypes)

Food_ID                   int64
Food_Name                object
Quantity                  int64
Expiry_Date      datetime64[ns]
Provider_ID               int64
Provider_Type            object
Location                 object
Food_Type                object
Meal_Type                object
dtype: object
Claim_ID                int64
Food_ID                 int64
Receiver_ID             int64
Status                 object
Timestamp      datetime64[ns]
dtype: object


In [3]:
!pip install psycopg2-binary sqlalchemy

In [4]:
#CREATE DATABASE
import psycopg2

conn=psycopg2.connect(
    host='localhost',
    database='postgres',
    user='postgres',
    password='POSTGRESQL'
)
conn.autocommit = True
cursor=conn.cursor()
cursor.execute("CREATE DATABASE food_waste;")
print("Database 'food_waste' created successfully.")
cursor.close()
conn.close()

DuplicateDatabase: database "food_waste" already exists


In [5]:
#CREATE TABLE AND INSERT VALUES FOR providers
import psycopg2

conn= psycopg2.connect(
    host='localhost',
    database='food_waste',
    user='postgres',
    password='POSTGRESQL'
)
cursor=conn.cursor()

cursor.execute("""CREATE TABLE IF NOT EXISTS providers(Provider_ID INT primary key,Name TEXT,Type TEXT,Address TEXT,City TEXT,Contact TEXT);""")

provider_data=[tuple(row) for index , row in providers.iterrows()]
cursor.executemany("""INSERT INTO providers(Provider_ID,Name,Type,Address,City,Contact)values(%s,%s,%s,%s,%s,%s)""",provider_data)
conn.commit()

print('providers table has created and value inserted')
cursor.close()
conn.close()


providers table has created and value inserted


In [6]:
#CREATE TABLE AND INSERT VALUES FOR receivers
import psycopg2

conn= psycopg2.connect(
    host='localhost',
    database='food_waste',
    user='postgres',
    password='POSTGRESQL'
)
cursor=conn.cursor()

cursor.execute("""CREATE TABLE IF NOT EXISTS receivers(Receiver_ID INT primary key,Name TEXT,Type TEXT,City TEXT,Contact TEXT);""")
receiver_data=[tuple(row) for index,row in receivers.iterrows()]
cursor.executemany("""INSERT INTO receivers(Receiver_ID,Name,Type,City,Contact)values(%s,%s,%s,%s,%s)""",receiver_data)
conn.commit()
cursor.close()
conn.close()
print('receivers table has created and value inserted')

receivers table has created and value inserted


In [7]:
#CREATE TABLE AND INSERT VALUES FOR Food_listing
import psycopg2

conn= psycopg2.connect(
    host='localhost',
    database='food_waste',
    user='postgres',
    password='POSTGRESQL'
)
cursor=conn.cursor()
cursor.execute("""CREATE TABLE IF NOT EXISTS food_listings(Food_ID INT primary key,Food_Name TEXT,Quantity	INT,Expiry_Date	DATE,Provider_ID INT,Provider_Type TEXT,Location TEXT,Food_Type TEXT,Meal_Type TEXT);""")
food_listing_data=[tuple(row) for index, row in food_listings.iterrows()]
cursor.executemany("""INSERT INTO food_listings(Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type)values(%s,%s,%s,%s,%s,%s,%s,%s,%s)""",food_listing_data)
conn.commit()
cursor.close()
conn.close()
print('food_listing table has created and value inserted')

food_listing table has created and value inserted


In [8]:
#CREATE TABLE AND INSERT VALUES FOR claimes
import psycopg2

conn= psycopg2.connect(
    host='localhost',
    database='food_waste',
    user='postgres',
    password='POSTGRESQL'
)
cursor=conn.cursor()
cursor.execute("""CREATE TABLE IF NOT EXISTS claims(Claim_ID INT primary key,Food_ID INT,Receiver_ID INT,Status	TEXT,Timestamp TIMESTAMP);""")
claim=[tuple(row) for index ,row in claims.iterrows()]
cursor.executemany("""INSERT INTO claims(Claim_ID,Food_ID,Receiver_ID,Status,Timestamp)values(%s,%s,%s,%s,%s)""",claim)
conn.commit()
cursor.close()
conn.close()
print("claims table has created and value inserted")

claims table has created and value inserted


In [10]:
#query 1
#How many food providers and receivers are there in each city?

import psycopg2
import pandas as pd

conn= psycopg2.connect(
    host='localhost',
    database='food_waste',
    user='postgres',
    password='POSTGRESQL'
)
cursor=conn.cursor()
query_1 = """
SELECT 
    COALESCE(p.city, r.city) AS city,
    COALESCE(p.num_providers, 0) AS num_providers,
    COALESCE(r.num_receivers, 0) AS num_receivers
FROM 
    (SELECT city, COUNT(*) AS num_providers FROM providers GROUP BY city) p
FULL OUTER JOIN 
    (SELECT city, COUNT(*) AS num_receivers FROM receivers GROUP BY city) r
ON p.city = r.city
ORDER BY city;
"""
cursor.execute(query_1)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["City", "Num_Providers", "Num_Receivers"])

# Show the table
df

,City,Num_Providers,Num_Receivers
0,Aaronshire,0,1
1,Adambury,1,0
2,Adamland,0,1
3,Adamsview,1,0
4,Adamsville,1,0
...,...,...,...
1873,Yatesside,1,0
1874,Youngchester,1,0
1875,Zacharyview,0,1
1876,Zimmermanton,1,0


In [11]:
#Which type of food provider (restaurant, grocery store, etc.) contributes the most food?
query_2= """
-- Find the provider type that contributes the most food quantity
SELECT 
    provider_type,
    SUM(quantity) AS total_quantity
FROM food_listings
GROUP BY provider_type
ORDER BY total_quantity DESC
LIMIT 3;
"""

cursor.execute(query_2)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["Provider_Type", "Total_Quantity"])
df


,Provider_Type,Total_Quantity
0,Restaurant,6923
1,Supermarket,6696
2,Catering Service,6116


In [12]:
#What is the contact information of food providers in a specific city?

city_name = 'New Jessica'

query_3 = f"""
SELECT 
    name,
    type,
    contact
FROM providers
WHERE city = '{city_name}';
"""

cursor.execute(query_3)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["Name", "Type", "Contact"])
df


,Name,Type,Contact
0,Gonzales-Cochran,Supermarket,+1-600-220-0480


In [16]:
#Which receivers have claimed the most food?

conn.rollback()

query_4= """
SELECT 
    r.name AS receiver_name,
    COUNT(c.claim_id) AS total_claims
FROM claims c
JOIN receivers r ON c.receiver_id = r.receiver_id
WHERE c.status = 'Completed'
GROUP BY r.name
ORDER BY total_claims DESC LIMIT 5;"""

cursor.execute(query_4)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["Receiver_Name", "Total_Claims"])
df

,Receiver_Name,Total_Claims
0,Timothy Garrett,3
1,Alexandra Owens,3
2,Derek Potter,3
3,James Miller,3
4,Alvin West,2


In [18]:
#What is the total quantity of food available from all providers?

query_5= """
SELECT 
    SUM(quantity) AS total_food_quantity
FROM food_listings;
"""

cursor.execute(query_5)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["Total_Food_Quantity"])
df


,Total_Food_Quantity
0,25794


In [19]:
#Which city has the highest number of food listings?

query_6= """
SELECT 
    p.city,
    SUM(f.quantity) AS total_quantity,
    COUNT(f.Food_ID) as toal_listings
FROM 
    food_listings f
JOIN 
    providers p ON f.provider_id = p.provider_id
GROUP BY 
    p.city
ORDER BY 
    total_quantity DESC
LIMIT 5;

"""

cursor.execute(query_6)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["City", "total_quantiy","total_listing"])
df


,City,total_quantiy,total_listing
0,South Kathryn,179,6
1,Jonathanstad,169,4
2,New Carol,167,6
3,North Keith,158,4
4,Jimmyberg,150,5


In [20]:
#What are the most commonly available food types?

query_7= """
SELECT 
    food_type,
    COUNT(*) AS total_listings
FROM food_listings
GROUP BY food_type
ORDER BY total_listings DESC LIMIT 3;
"""

cursor.execute(query_7)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["Food_Type", "Total_Listings"])
df


,Food_Type,Total_Listings
0,Vegetarian,336
1,Vegan,334
2,Non-Vegetarian,330


In [21]:
#How many food claims have been made for each food item?

query_8= """
-- Count how many claims have been made for each food item
SELECT 
    f.food_name,
    c.food_id,
    COUNT(c.claim_id) AS total_claims
FROM claims c
JOIN food_listings f ON c.food_id = f.food_id
GROUP BY c.food_id, f.food_name
ORDER BY total_claims DESC LIMIT 5;
"""

cursor.execute(query_8)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["Food_Name", "Food_ID", "Total_Claims"])
df


,Food_Name,Food_ID,Total_Claims
0,Fish,548,5
1,Soup,463,5
2,Chicken,486,5
3,Fish,832,4
4,Dairy,251,4


In [22]:
# Which provider has had the highest number of successful food claims?

conn.rollback()

query_9 = """
SELECT 
    p.name AS provider_name,
    COUNT(c.claim_id) AS successful_claims
FROM claims c
JOIN food_listings f ON c.food_id = f.food_id
JOIN providers p ON f.provider_id = p.provider_id
WHERE c.status = 'Completed'
GROUP BY p.name
ORDER BY successful_claims DESC
LIMIT 2;
"""

cursor.execute(query_9)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["Provider_Name", "Successful_Claims"])
df


,Provider_Name,Successful_Claims
0,Barry Group,5
1,Butler-Richardson,4


In [23]:
#What percentage of food claims are completed vs. pending vs. canceled?

query_10= """
SELECT 
    Status,
    COUNT(*) AS total_claims,
    ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM claims), 2) AS percentage
FROM claims
GROUP BY Status;

"""

cursor.execute(query_10)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["Status", "Count", "Percentage"])
df


,Status,Count,Percentage
0,Completed,339,33.90
1,Cancelled,336,33.60
2,Pending,325,32.50


In [24]:
#What is the average quantity of food claimed per receiver?

query_11= """
SELECT 
    r.name AS receiver_name,
    AVG(f.quantity) AS avg_quantity_claimed
FROM claims c
JOIN food_listings f ON c.food_id = f.food_id
JOIN receivers r ON c.receiver_id = r.receiver_id
GROUP BY r.name
ORDER BY avg_quantity_claimed DESC LIMIT 20;
"""

cursor.execute(query_11)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["Receiver_Name", "Avg_Quantity_Claimed"])
df


,Receiver_Name,Avg_Quantity_Claimed
0,Peggy Knight,50.0000000000000000
1,Thomas Villanueva,50.0000000000000000
2,Nancy Jones,50.0000000000000000
3,Daniel Williams,50.0000000000000000
4,Nancy Silva,50.0000000000000000
5,Lisa Pitts,50.0000000000000000
6,Christopher Wright,50.0000000000000000
7,Angel Adams,49.0000000000000000
8,Melissa Little,49.0000000000000000
9,Cassandra Yoder,49.0000000000000000


In [25]:
#Which meal type (breakfast, lunch, dinner, snacks) is claimed the most?

query_12= """
-- Find which meal type is claimed the most
SELECT 
    f.meal_type,
    COUNT(c.claim_id) AS total_claims
FROM claims c
JOIN food_listings f ON c.food_id = f.food_id
WHERE c.status = 'Completed'
GROUP BY f.meal_type
ORDER BY total_claims DESC
LIMIT 1;
"""

cursor.execute(query_12)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["Meal_Type", "Total_Claims"])
df


,Meal_Type,Total_Claims
0,Breakfast,95


In [26]:
#What is the total quantity of food donated by each provider?

query_13= """
-- Total quantity of food donated by each provider
SELECT 
    p.name AS provider_name,
    SUM(f.quantity) AS total_quantity_donated
FROM food_listings f
JOIN providers p ON f.provider_id = p.provider_id
GROUP BY p.name
ORDER BY total_quantity_donated DESC;
"""

cursor.execute(query_13)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["Provider_Name", "Total_Quantity_Donated"])
df


,Provider_Name,Total_Quantity_Donated
0,Miller Inc,217
1,Barry Group,179
2,"Evans, Wright and Mitchell",158
3,Smith Group,150
4,Campbell LLC,145
...,...,...
623,Norris-Townsend,1
624,Mcgee PLC,1
625,Gonzales-Moore,1
626,Reyes and Sons,1


In [27]:
#Which food types are listed the most but rarely claimed?

Addition_query_1 = """
SELECT 
    f.Food_Type,
    COUNT(DISTINCT f.Food_ID) AS total_listings,
    COUNT(DISTINCT c.Claim_ID) AS total_claims,
    ROUND(100.0 * COUNT(DISTINCT c.Claim_ID) / COUNT(DISTINCT f.Food_ID), 2) AS claim_rate
FROM food_listings f
LEFT JOIN claims c ON f.Food_ID = c.Food_ID
GROUP BY f.Food_Type
ORDER BY claim_rate ASC;
"""
cursor.execute(Addition_query_1)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["Food_Type", "Total_Listings", "Total_Claims", "Claim_Rate"])
df

,Food_Type,Total_Listings,Total_Claims,Claim_Rate
0,Vegan,334,319,95.51
1,Non-Vegetarian,330,331,100.30
2,Vegetarian,336,350,104.17


In [2]:
#Which providers have the most unclaimed food (by quantity)?

import psycopg2
import pandas as pd

conn= psycopg2.connect(
    host='localhost',
    database='food_waste',
    user='postgres',
    password='POSTGRESQL'
)
cursor=conn.cursor()

new_addition_query_2 = """
SELECT
    p.Name AS Provider_Name,
    SUM(fl.Quantity) AS Total_Unclaimed_Quantity
FROM food_listings fl
JOIN providers p ON fl.Provider_ID = p.Provider_ID
WHERE fl.Food_ID NOT IN (SELECT food_id FROM claims WHERE status = 'Completed')
GROUP BY p.Name
ORDER BY Total_Unclaimed_Quantity DESC;
"""
cursor.execute(new_addition_query_2)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["Provider_Name","Total_Unclaimed_Quantity"])
df

,Provider_Name,Total_Unclaimed_Quantity
0,Miller Inc,173
1,Smith Group,150
2,Ruiz-Oneal,140
3,"Evans, Wright and Mitchell",132
4,Shepherd and Sons,116
...,...,...
495,Reyes and Sons,1
496,Gonzales-Moore,1
497,Fernandez-Bridges,1
498,Norris-Townsend,1


In [7]:
#What quantities of food are typically received by different types of receivers?

conn.rollback()

new_addition_query_3="""
SELECT
    r.Type AS Receiver_Type,
    SUM(fl.Quantity) AS Total_Quantity_Received
FROM
    claims c
JOIN
    food_listings fl ON c.Food_ID = fl.Food_ID
JOIN
    receivers r ON c.Receiver_ID = r.Receiver_ID
WHERE
    c.Status = 'Completed'
GROUP BY
    r.Type
ORDER BY
    Total_Quantity_Received DESC;
"""
cursor.execute(new_addition_query_3)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["Receiver_Type", "Total_Quantity_Received"])
df

,Receiver_Type,Total_Quantity_Received
0,Charity,2538
1,NGO,2358
2,Shelter,2056
3,Individual,1821


In [8]:
#Which day of the week sees the most unclaimed food?

conn.rollback()

new_addition_query_4 = """
SELECT 
    TO_CHAR(f.Expiry_Date, 'Day') AS Day_Of_Week,
    SUM(f.Quantity) - COUNT(c.Claim_ID) AS Unclaimed_Quantity
FROM food_listings f
LEFT JOIN claims c ON f.Food_ID = c.Food_ID AND c.Status = 'Completed'
GROUP BY Day_Of_Week
ORDER BY Unclaimed_Quantity DESC;
"""

cursor.execute(new_addition_query_4)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["Day_Of_Week", "Unclaimed_Quantity"])
df



,Day_Of_Week,Unclaimed_Quantity
0,Sunday,5050
1,Monday,4232
2,Saturday,4076
3,Thursday,3617
4,Friday,3485
5,Tuesday,3438
6,Wednesday,3016


In [9]:
#How many claims are made after the food expired?

new_addition_query_5 = """
SELECT 
    COUNT(*) AS late_claims
FROM claims c
JOIN food_listings f ON c.Food_ID = f.Food_ID
WHERE c.Timestamp > f.Expiry_Date;
"""
cursor.execute(new_addition_query_5)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["Late_Claims"])
df

,Late_Claims
0,55


In [10]:
#Which provider types donate the most total food quantity?

new_addition_query_6 = """
SELECT 
    p.Type AS Provider_Type,
    SUM(f.Quantity) AS Total_Quantity
FROM providers p
JOIN food_listings f ON p.Provider_ID = f.Provider_ID
GROUP BY p.Type
ORDER BY Total_Quantity DESC;
"""
cursor.execute(new_addition_query_6)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["Provider_Type", "Total_Quantity"])
df

,Provider_Type,Total_Quantity
0,Restaurant,6923
1,Supermarket,6696
2,Catering Service,6116
3,Grocery Store,6059


In [11]:
#Which receiver city receives the most total claimed quantity?
conn.rollback()

new_addition_query_7 = """
SELECT 
    r.City,
    SUM(f.Quantity) AS Total_Claimed
FROM claims c
JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
JOIN food_listings f ON c.Food_ID = f.Food_ID
WHERE c.Status = 'Completed'
GROUP BY r.City
ORDER BY Total_Claimed DESC;
"""

cursor.execute(new_addition_query_7)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["City", "Total_Claimed"])
df

,City,Total_Claimed
0,South Laurachester,99
1,Cruzborough,89
2,East Michael,84
3,Biancaton,83
4,Calebview,82
...,...,...
284,East Janetstad,2
285,Mariefurt,2
286,East Travis,1
287,South Nicoleberg,1


In [12]:
#Which meal type has the highest waste (listed but not claimed)?

conn.rollback()

new_addition_query_8 = """
SELECT 
    f.Meal_Type,
    SUM(f.Quantity) - COALESCE(SUM(CASE WHEN c.Status = 'Completed' THEN f.Quantity ELSE 0 END), 0) AS Unclaimed_Quantity
FROM food_listings f
LEFT JOIN claims c ON f.Food_ID = c.Food_ID
GROUP BY f.Meal_Type
ORDER BY Unclaimed_Quantity DESC;
"""

cursor.execute(new_addition_query_8)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["Meal_Type", "Unclaimed_Quantity"])
df


,Meal_Type,Unclaimed_Quantity
0,Breakfast,7007
1,Snacks,6936
2,Lunch,6449
3,Dinner,5954


In [13]:
#Which food items were listed the most but never claimed?

new_addition_query_9 = """
SELECT 
    f.Food_Name,
    COUNT(f.Food_ID) AS total_listed
FROM food_listings f
LEFT JOIN claims c ON f.Food_ID = c.Food_ID
WHERE c.Claim_ID IS NULL
GROUP BY f.Food_Name
ORDER BY total_listed DESC
LIMIT 5;
"""
cursor.execute(new_addition_query_9)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["Food_Name", "Unclaimed_Listings"])
df

,Food_Name,Unclaimed_Listings
0,Pasta,41
1,Soup,39
2,Dairy,38
3,Salad,36
4,Chicken,35


In [14]:
#Which providers list the most food items that end up unclaimed?

new_addition_query_10 = """
SELECT 
    p.Name AS Provider_Name,
    COUNT(DISTINCT f.Food_ID) AS total_listings,
    COUNT(DISTINCT c.Claim_ID) AS total_claims,
    COUNT(DISTINCT f.Food_ID) - COUNT(DISTINCT c.Claim_ID) AS unclaimed_listings
FROM providers p
JOIN food_listings f ON p.Provider_ID = f.Provider_ID
LEFT JOIN claims c ON f.Food_ID = c.Food_ID
GROUP BY p.Name
ORDER BY unclaimed_listings DESC
LIMIT 5;
"""
cursor.execute(new_addition_query_10)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["Provider_Name", "Total_Listings", "Total_Claims", "Unclaimed_Listings"])
df


,Provider_Name,Total_Listings,Total_Claims,Unclaimed_Listings
0,Smith Group,5,0,5
1,Smith-Villegas,3,0,3
2,Daniels-Kim,5,2,3
3,Ruiz-Oneal,4,1,3
4,Schneider-Suarez,3,0,3
